<a href="https://colab.research.google.com/github/martinpdab/asapa/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sklearn

In [2]:
import numpy as np
import pandas as pd

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
URL = 'https://raw.githubusercontent.com/martinpdab/asapa/main/dataset.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,Tempat Kejadian,Jenis Kekerasan,Banyak Kekerasan yang dialami,Banyak Korban Kekerasan,Usia Korban,Hubungan Pelaku dengan Korban,Mitigasi Resiko
0,rumah-tangga,fisik,1,1,balita,orang-tua,3
1,rumah-tangga,fisik,1,1,balita,orang-tua,3
2,rumah-tangga,fisik,1,1,balita,orang-tua,3
3,rumah-tangga,fisik,1,1,balita,orang-tua,3
4,rumah-tangga,fisik,1,1,balita,orang-tua,3


In [4]:
from sklearn.utils import shuffle
df = shuffle(dataframe)


In [5]:
df.head()

,Tempat Kejadian,Jenis Kekerasan,Banyak Kekerasan yang dialami,Banyak Korban Kekerasan,Usia Korban,Hubungan Pelaku dengan Korban,Mitigasi Resiko
2409,rumah-tangga,psikis,1,1,dewasa,tetangga,1
592,rumah-tangga,fisik,1,1,dewasa,suami/istri,3
1170,rumah-tangga,seksual,1,1,remaja,orang-tua,3
82,rumah-tangga,fisik,1,1,balita,keluarga/saudara,3
2939,rumah-tangga,lainnya,1,1,remaja,orang-tua,1


In [6]:
train, test = train_test_split(df, test_size=0.1)
print(len(train), 'train examples')
print(len(test), 'test examples')

4697 train examples
522 test examples


In [7]:
def df_to_dataset(df, shuffle=True, batch_size=32):
  df = df.copy()
  labels = df.pop('Mitigasi Resiko')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  return ds

In [8]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [9]:
for feature_batch, label_batch in train_ds.take(1):
  print('Parameter:', list(feature_batch.keys()))
  print('Tempat Kejadian:', feature_batch['Tempat Kejadian'])
  print('Jenis Kekerasan:', feature_batch['Jenis Kekerasan'])
  print('Banyak Kekerasan:', feature_batch['Banyak Kekerasan yang dialami'])
  print('Banyak Korban:', feature_batch['Banyak Korban Kekerasan'])
  print('Usia Korban:', feature_batch['Usia Korban'])
  print('Hubungan Pelaku dengan Korban:', feature_batch['Hubungan Pelaku dengan Korban'])
  print('Label:', label_batch )

Parameter: ['Tempat Kejadian', 'Jenis Kekerasan', 'Banyak Kekerasan yang dialami', 'Banyak Korban Kekerasan', 'Usia Korban', 'Hubungan Pelaku dengan Korban']
Tempat Kejadian: tf.Tensor(
[b'rumah-tangga' b'fasilitas-umum' b'fasilitas-umum' b'fasilitas-umum'
 b'rumah-tangga'], shape=(5,), dtype=string)
Jenis Kekerasan: tf.Tensor([b'fisik' b'fisik' b'seksual' b'lainnya' b'psikis'], shape=(5,), dtype=string)
Banyak Kekerasan: tf.Tensor([1 2 1 1 1], shape=(5,), dtype=int64)
Banyak Korban: tf.Tensor([1 1 1 1 1], shape=(5,), dtype=int64)
Usia Korban: tf.Tensor([b'dewasa' b'dewasa' b'dewasa' b'dewasa' b'dewasa'], shape=(5,), dtype=string)
Hubungan Pelaku dengan Korban: tf.Tensor(
[b'keluarga/saudara' b'suami/istri' b'keluarga/saudara' b'lainnya'
 b'majikan'], shape=(5,), dtype=string)
Label: tf.Tensor([2 3 2 2 1], shape=(5,), dtype=int64)


In [15]:
feature_columns = []

tempat_kejadian = tf.feature_column.categorical_column_with_vocabulary_list(
                'Tempat Kejadian', ['rumah-tangga', 'tempat-kerja', 'sekolah', 'fasilitas-umum', 'lainnya'])
tempat_one_hot = tf.feature_column.indicator_column(tempat_kejadian)
feature_columns.append(tempat_one_hot)

jenis_kekerasan = tf.feature_column.categorical_column_with_vocabulary_list(
    'Jenis Kekerasan', ['fisik', 'seksual', 'psikis', 'penelantaran', 'lainnya', 'eksploitasi', 'traficking'])
jenis_one_hot = tf.feature_column.indicator_column(jenis_kekerasan)
feature_columns.append(jenis_one_hot)